# Chapter 6: Server vs Client Components

Welcome to one of the most important chapters in this guide! Understanding Server Components and Client Components is fundamental to mastering Next.js. This represents a major shift in how we think about building React applications.

By the end of this chapter, you'll have a complete understanding of the mental model shift, know exactly when to use Server or Client Components, and be able to compose them effectively to build performant applications.

## 6.1 Understanding the Mental Model Shift

### The Traditional React Model

In traditional React development (before Next.js Server Components), all components rendered on the client. When a user visited your page:

1. The browser downloaded the JavaScript bundle
2. React hydrated the page
3. Components rendered on the client
4. Data was fetched from APIs (usually via `useEffect`)
5. The UI updated with the fetched data

**This approach had several limitations:**

- **Large JavaScript bundles**: All component code was sent to the client
- **Hydration complexity**: The server and client needed to stay in sync
- **Security concerns**: Sensitive logic ran on the client
- **Performance overhead**: More JavaScript meant slower initial loads

### The Next.js Server Component Model

Next.js introduces a fundamental shift: components can now run on the server by default. This changes everything:

**Server Components:**
- Render on the server
- Don't send JavaScript to the client (only HTML is sent)
- Can access server resources directly (databases, file system, APIs)
- Reduce the JavaScript bundle size
- Improve performance and SEO

**Client Components:**
- Render on the client (traditional React behavior)
- Require the "use client" directive
- Have access to browser APIs and user interactions
- Can use hooks like useState, useEffect, etc.
- Are hydrated like traditional React components

### The Hybrid Approach

Next.js allows you to mix Server and Client Components seamlessly:

```tsx
// Server Component (default)
export default function ServerComponentExample() {
  // This code runs on the server
  const data = fetchFromDatabase(); // Direct database access
  
  return (
    <div>
      <h1>Server Rendered Content</h1>
      <ClientButton /> {/* Client Component mixed in */}
    </div>
  );
}

// Client Component
"use client";

import { useState } from 'react';

function ClientButton() {
  const [count, setCount] = useState(0);
  
  return (
    <button onClick={() => setCount(count + 1)}>
      Clicks: {count}
    </button>
  );
}
```

### Visualizing the Difference

**Traditional Client-Side Rendering:**

```
User Request → HTML + JavaScript Bundle → Browser
                          ↓
                    React Hydration
                          ↓
                   Client Rendering
                          ↓
                    Data Fetching
                          ↓
                     UI Update
```

**Next.js Server Component Model:**

```
User Request → Server Components Render → HTML + Minimal JavaScript
                                                ↓
                                          Browser (hydrates only Client Components)
                                                ↓
                                      Interactive UI Ready
```

### Benefits of the Mental Model Shift

1. **Performance**: Server Components don't send JavaScript to the client, reducing bundle sizes

2. **Security**: Sensitive operations stay on the server where they belong

3. **Simplified Data Fetching**: No complex `useEffect` patterns for data fetching

4. **Better SEO**: Search engines get fully rendered HTML immediately

5. **Developer Experience**: Direct access to server resources in components

6. **Automatic Code Splitting**: Only Client Components are included in the bundle

### The Default: Server Components

In Next.js App Router, **all components are Server Components by default**. You need to explicitly mark components as Client Components when they need client-side features.

```tsx
// This is a Server Component by default
export default function MyComponent() {
  // ✅ Can fetch data directly
  const users = await db.users.findMany();
  
  // ✅ Can use server-side APIs
  const file = fs.readFileSync('data.json');
  
  // ❌ Cannot use hooks
  // const [count, setCount] = useState(0); // Error!
  
  // ❌ Cannot use browser APIs
  // const width = window.innerWidth; // Error!
  
  return <div>{users.length} users</div>;
}
```

## 6.2 Server Components by Default

Server Components are the default in Next.js, and understanding how they work is crucial for building efficient applications.

### What Server Components Can Do

Server Components have powerful capabilities:

```tsx
// app/users/page.tsx (Server Component)

import { db } from '@/lib/db';

export default async function UsersPage() {
  // 1. Direct database access
  const users = await db.user.findMany({
    include: {
      posts: true,
    },
  });

  // 2. Access to server-side APIs
  const secretData = process.env.SECRET_API_KEY;

  // 3. File system access
  import { readFile } from 'fs/promises';
  const data = await readFile('config.json', 'utf-8');

  // 4. Internal API calls (no CORS issues)
  const response = await fetch('https://api.internal.com/data', {
    cache: 'force-cache',
  });
  const apiData = await response.json();

  // 5. Complex server-side computations
  const analytics = calculateComplexAnalytics(users);

  return (
    <div className="container mx-auto px-4 py-8">
      <h1 className="text-3xl font-bold text-gray-900 mb-6">
        Users Dashboard
      </h1>

      <div className="grid grid-cols-1 md:grid-cols-3 gap-6 mb-8">
        <StatCard title="Total Users" value={users.length} />
        <StatCard title="Active Posts" value={users.reduce((sum, u) => sum + u.posts.length, 0)} />
        <StatCard title="Analytics Score" value={analytics.score} />
      </div>

      <div className="bg-white rounded-lg shadow-sm">
        <table className="w-full">
          <thead className="bg-gray-50">
            <tr>
              <th className="px-6 py-3 text-left text-xs font-medium text-gray-500 uppercase">
                Name
              </th>
              <th className="px-6 py-3 text-left text-xs font-medium text-gray-500 uppercase">
                Email
              </th>
              <th className="px-6 py-3 text-left text-xs font-medium text-gray-500 uppercase">
                Posts
              </th>
            </tr>
          </thead>
          <tbody className="divide-y divide-gray-200">
            {users.map((user) => (
              <tr key={user.id}>
                <td className="px-6 py-4">{user.name}</td>
                <td className="px-6 py-4">{user.email}</td>
                <td className="px-6 py-4">{user.posts.length}</td>
              </tr>
            ))}
          </tbody>
        </table>
      </div>
    </div>
  );
}

function StatCard({ title, value }: { title: string; value: number }) {
  return (
    <div className="bg-white p-6 rounded-lg shadow-sm">
      <h3 className="text-sm font-medium text-gray-500 mb-2">{title}</h3>
      <p className="text-2xl font-bold text-gray-900">{value}</p>
    </div>
  );
}
```

### What Server Components Cannot Do

Server Components have limitations (by design):

```tsx
// Server Component limitations

export default function LimitedServerComponent() {
  // ❌ Cannot use React hooks
  // useState, useEffect, useContext, useReducer, etc.
  // const [count, setCount] = useState(0); // Error!

  // ❌ Cannot use browser APIs
  // window, document, localStorage, sessionStorage, etc.
  // const width = window.innerWidth; // Error!
  // const stored = localStorage.getItem('key'); // Error!

  // ❌ Cannot define event handlers
  // <button onClick={handleClick}>Click</button> // Error!

  // ❌ Cannot use React lifecycle methods
  // componentDidMount, componentDidUpdate, etc.

  // ❌ Cannot use the "use" directive
  // Only Client Components can use "use client"

  // ✅ Everything else works fine
  return <div>Server rendered content</div>;
}
```

### Server Components and Streaming

Server Components support streaming, which allows parts of your page to load progressively:

```tsx
// app/dashboard/page.tsx

import { Suspense } from 'react';
import { UserStats } from '@/components/UserStats';
import { RecentActivity } from '@/components/RecentActivity';
import { RevenueChart } from '@/components/RevenueChart';

export default function DashboardPage() {
  return (
    <div className="container mx-auto px-4 py-8">
      <h1 className="text-3xl font-bold text-gray-900 mb-8">
        Dashboard
      </h1>

      <div className="grid grid-cols-1 lg:grid-cols-2 gap-8">
        {/* User stats stream in independently */}
        <Suspense fallback={<StatsLoadingSkeleton />}>
          <UserStats />
        </Suspense>

        {/* Recent activity streams in independently */}
        <Suspense fallback={<ActivityLoadingSkeleton />}>
          <RecentActivity />
        </Suspense>

        {/* Revenue chart streams in independently */}
        <div className="lg:col-span-2">
          <Suspense fallback={<ChartLoadingSkeleton />}>
            <RevenueChart />
          </Suspense>
        </div>
      </div>
    </div>
  );
}

function StatsLoadingSkeleton() {
  return (
    <div className="bg-white p-6 rounded-lg shadow-sm animate-pulse">
      <div className="h-6 bg-gray-200 rounded w-1/3 mb-4"></div>
      <div className="space-y-3">
        <div className="h-4 bg-gray-200 rounded"></div>
        <div className="h-4 bg-gray-200 rounded w-2/3"></div>
      </div>
    </div>
  );
}

function ActivityLoadingSkeleton() {
  return (
    <div className="bg-white p-6 rounded-lg shadow-sm animate-pulse">
      <div className="h-6 bg-gray-200 rounded w-1/3 mb-4"></div>
      <div className="space-y-3">
        {[1, 2, 3].map((i) => (
          <div key={i} className="flex items-center space-x-3">
            <div className="h-10 w-10 bg-gray-200 rounded-full"></div>
            <div className="flex-1 space-y-2">
              <div className="h-4 bg-gray-200 rounded w-3/4"></div>
              <div className="h-3 bg-gray-200 rounded w-1/2"></div>
            </div>
          </div>
        ))}
      </div>
    </div>
  );
}

function ChartLoadingSkeleton() {
  return (
    <div className="bg-white p-6 rounded-lg shadow-sm animate-pulse">
      <div className="h-6 bg-gray-200 rounded w-1/3 mb-4"></div>
      <div className="h-64 bg-gray-200 rounded"></div>
    </div>
  );
}
```

### Server Components and Caching

Server Components work seamlessly with Next.js's caching system:

```tsx
// app/products/page.tsx

export default async function ProductsPage() {
  // This fetch is cached by default
  const products = await fetch('https://api.example.com/products', {
    cache: 'force-cache',
  }).then(res => res.json());

  // This fetch is revalidated every 60 seconds
  const featured = await fetch('https://api.example.com/featured', {
    next: { revalidate: 60 },
  }).then(res => res.json());

  // This fetch is never cached
  const realtime = await fetch('https://api.example.com/realtime', {
    cache: 'no-store',
  }).then(res => res.json());

  return (
    <div className="container mx-auto px-4 py-8">
      <div className="grid grid-cols-1 md:grid-cols-2 lg:grid-cols-3 gap-6">
        {products.map((product: any) => (
          <ProductCard key={product.id} product={product} />
        ))}
      </div>
    </div>
  );
}

function ProductCard({ product }: { product: any }) {
  return (
    <div className="bg-white p-6 rounded-lg shadow-sm">
      <h3 className="font-semibold text-gray-900 mb-2">{product.name}</h3>
      <p className="text-gray-600 mb-4">{product.description}</p>
      <p className="text-xl font-bold text-gray-900">${product.price}</p>
    </div>
  );
}
```

## 6.3 Client Components with "use client"

Client Components are the traditional React components you're familiar with. They require the "use client" directive at the top of the file.

### When to Use Client Components

Use Client Components when you need:

1. **Interactive Features**: Click handlers, form submissions, etc.
2. **Browser APIs**: window, document, localStorage, etc.
3. **React Hooks**: useState, useEffect, useContext, etc.
4. **State Management**: Component-local state
5. **Event Listeners**: Custom event handling

### Basic Client Component

```tsx
// app/components/Counter.tsx
"use client";

import { useState } from 'react';

export default function Counter() {
  const [count, setCount] = useState(0);

  return (
    <div className="flex items-center space-x-4">
      <button
        onClick={() => setCount(count - 1)}
        className="px-4 py-2 bg-gray-200 rounded hover:bg-gray-300"
      >
        -
      </button>
      
      <span className="text-xl font-bold">{count}</span>
      
      <button
        onClick={() => setCount(count + 1)}
        className="px-4 py-2 bg-gray-200 rounded hover:bg-gray-300"
      >
        +
      </button>
    </div>
  );
}
```

### Client Component with Browser APIs

```tsx
// app/components/WindowWidth.tsx
"use client";

import { useState, useEffect } from 'react';

export default function WindowWidth() {
  const [width, setWidth] = useState(0);

  useEffect(() => {
    // Access browser API
    setWidth(window.innerWidth);

    const handleResize = () => setWidth(window.innerWidth);

    window.addEventListener('resize', handleResize);

    // Cleanup
    return () => window.removeEventListener('resize', handleResize);
  }, []);

  return (
    <div className="bg-blue-50 p-4 rounded-lg">
      <p className="text-sm text-gray-600">
        Window width: <span className="font-bold text-blue-600">{width}px</span>
      </p>
    </div>
  );
}
```

### Client Component with Form Handling

```tsx
// app/components/ContactForm.tsx
"use client";

import { useState, FormEvent } from 'react';

interface FormData {
  name: string;
  email: string;
  message: string;
}

export default function ContactForm() {
  const [formData, setFormData] = useState<FormData>({
    name: '',
    email: '',
    message: '',
  });
  const [isSubmitting, setIsSubmitting] = useState(false);
  const [submitStatus, setSubmitStatus] = useState<'idle' | 'success' | 'error'>('idle');

  const handleSubmit = async (e: FormEvent) => {
    e.preventDefault();
    setIsSubmitting(true);
    setSubmitStatus('idle');

    try {
      const response = await fetch('/api/contact', {
        method: 'POST',
        headers: { 'Content-Type': 'application/json' },
        body: JSON.stringify(formData),
      });

      if (response.ok) {
        setSubmitStatus('success');
        setFormData({ name: '', email: '', message: '' });
      } else {
        setSubmitStatus('error');
      }
    } catch (error) {
      setSubmitStatus('error');
    } finally {
      setIsSubmitting(false);
    }
  };

  const handleChange = (e: React.ChangeEvent<HTMLInputElement | HTMLTextAreaElement>) => {
    setFormData(prev => ({
      ...prev,
      [e.target.name]: e.target.value,
    }));
  };

  return (
    <form onSubmit={handleSubmit} className="space-y-6">
      <div>
        <label htmlFor="name" className="block text-sm font-medium text-gray-700 mb-2">
          Name
        </label>
        <input
          type="text"
          id="name"
          name="name"
          value={formData.name}
          onChange={handleChange}
          required
          className="w-full px-4 py-2 border border-gray-300 rounded-lg focus:ring-2 focus:ring-blue-500 focus:border-transparent"
        />
      </div>

      <div>
        <label htmlFor="email" className="block text-sm font-medium text-gray-700 mb-2">
          Email
        </label>
        <input
          type="email"
          id="email"
          name="email"
          value={formData.email}
          onChange={handleChange}
          required
          className="w-full px-4 py-2 border border-gray-300 rounded-lg focus:ring-2 focus:ring-blue-500 focus:border-transparent"
        />
      </div>

      <div>
        <label htmlFor="message" className="block text-sm font-medium text-gray-700 mb-2">
          Message
        </label>
        <textarea
          id="message"
          name="message"
          value={formData.message}
          onChange={handleChange}
          required
          rows={4}
          className="w-full px-4 py-2 border border-gray-300 rounded-lg focus:ring-2 focus:ring-blue-500 focus:border-transparent"
        />
      </div>

      <button
        type="submit"
        disabled={isSubmitting}
        className="w-full bg-blue-600 text-white px-6 py-3 rounded-lg hover:bg-blue-700 disabled:opacity-50 disabled:cursor-not-allowed transition-colors"
      >
        {isSubmitting ? 'Sending...' : 'Send Message'}
      </button>

      {submitStatus === 'success' && (
        <div className="bg-green-50 text-green-800 p-4 rounded-lg">
          Message sent successfully!
        </div>
      )}

      {submitStatus === 'error' && (
        <div className="bg-red-50 text-red-800 p-4 rounded-lg">
          Failed to send message. Please try again.
        </div>
      )}
    </form>
  );
}
```

### Client Component with Local Storage

```tsx
// app/components/ThemeToggle.tsx
"use client";

import { useState, useEffect } from 'react';

export default function ThemeToggle() {
  const [isDark, setIsDark] = useState(false);

  useEffect(() => {
    // Check local storage on mount
    const savedTheme = localStorage.getItem('theme');
    if (savedTheme === 'dark') {
      setIsDark(true);
      document.documentElement.classList.add('dark');
    }
  }, []);

  const toggleTheme = () => {
    setIsDark(!isDark);
    
    if (!isDark) {
      localStorage.setItem('theme', 'dark');
      document.documentElement.classList.add('dark');
    } else {
      localStorage.setItem('theme', 'light');
      document.documentElement.classList.remove('dark');
    }
  };

  return (
    <button
      onClick={toggleTheme}
      className="p-2 rounded-lg bg-gray-200 hover:bg-gray-300 transition-colors"
      aria-label="Toggle theme"
    >
      {isDark ? '🌙' : '☀️'}
    </button>
  );
}
```

## 6.4 When to Use Each Component Type

Choosing between Server and Client Components is a critical decision. Here's a comprehensive guide to help you make the right choice.

### Decision Tree

```
Does the component need interactivity?
├─ No → Use Server Component
└─ Yes → Does it need browser APIs?
        ├─ No → Can you extract interactivity to a child component?
        │       ├─ Yes → Use Server Component + Client child
        │       └─ No → Use Client Component
        └─ Yes → Use Client Component
```

### Server Component Checklist

Use Server Components when:

✅ **Displaying data fetched from a database or API**
```tsx
// Server Component - Perfect for data fetching
export default async function UserList() {
  const users = await db.user.findMany();
  return <UserTable users={users} />;
}
```

✅ **Rendering static or rarely-changing content**
```tsx
// Server Component - Great for static content
export default function AboutPage() {
  return (
    <div>
      <h1>About Us</h1>
      <p>We are a company...</p>
    </div>
  );
}
```

✅ **Keeping sensitive logic on the server**
```tsx
// Server Component - Sensitive operations stay on server
export default function AdminDashboard() {
  const sensitiveData = process.env.SECRET_KEY;
  // This code never reaches the client
  return <div>{/* Render admin content */}</div>;
}
```

✅ **Accessing server-side resources directly**
```tsx
// Server Component - Direct server resource access
export default function ConfigDisplay() {
  const config = JSON.parse(fs.readFileSync('config.json'));
  return <pre>{JSON.stringify(config, null, 2)}</pre>;
}
```

✅ **Improving performance by reducing bundle size**
```tsx
// Server Component - Reduces JavaScript sent to client
export default function LargeListComponent() {
  const items = generateLargeList(); // Runs on server
  return <List items={items} />;
}
```

✅ **SEO-critical pages**
```tsx
// Server Component - SEO optimized
export default async function ProductPage({ params }: { params: { id: string } }) {
  const product = await getProduct(params.id);
  
  return (
    <>
      <Head>
        <title>{product.name}</title>
        <meta name="description" content={product.description} />
      </Head>
      <ProductDetails product={product} />
    </>
  );
}
```

### Client Component Checklist

Use Client Components when:

✅ **Handling user interactions**
```tsx
// Client Component - Needs interactivity
"use client";

export default function InteractiveButton() {
  const [clicked, setClicked] = useState(false);
  return <button onClick={() => setClicked(true)}>Click me</button>;
}
```

✅ **Using browser APIs**
```tsx
// Client Component - Needs browser APIs
"use client";

export default function ScrollIndicator() {
  const [scrolled, setScrolled] = useState(false);

  useEffect(() => {
    const handleScroll = () => setScrolled(window.scrollY > 100);
    window.addEventListener('scroll', handleScroll);
    return () => window.removeEventListener('scroll', handleScroll);
  }, []);

  return scrolled ? <div className="top-bar">Scrolled!</div> : null;
}
```

✅ **Managing component-local state**
```tsx
// Client Component - Needs state management
"use client";

export default function Accordion({ items }: { items: { title: string; content: string }[] }) {
  const [openIndex, setOpenIndex] = useState(0);

  return (
    <div>
      {items.map((item, index) => (
        <div key={index}>
          <button onClick={() => setOpenIndex(index)}>{item.title}</button>
          {openIndex === index && <p>{item.content}</p>}
        </div>
      ))}
    </div>
  );
}
```

✅ **Using React hooks**
```tsx
// Client Component - Needs hooks
"use client";

import { usePathname } from 'next/navigation';

export default function Breadcrumbs() {
  const pathname = usePathname();
  const segments = pathname.split('/').filter(Boolean);

  return (
    <nav>
      {segments.map((segment, index) => (
        <span key={segment}>
          {index > 0 && ' / '}
          {segment}
        </span>
      ))}
    </nav>
  );
}
```

✅ **Listening to browser events**
```tsx
// Client Component - Needs event listeners
"use client";

export default function KeyboardShortcuts() {
  useEffect(() => {
    const handleKeyDown = (e: KeyboardEvent) => {
      if (e.key === 'Escape') {
        // Handle escape key
      }
    };

    window.addEventListener('keydown', handleKeyDown);
    return () => window.removeEventListener('keydown', handleKeyDown);
  }, []);

  return <div>Press Escape to close</div>;
}
```

### Mixed Approach: Best of Both Worlds

Often, the best approach is to use Server Components with Client Component children:

```tsx
// app/products/[id]/page.tsx (Server Component)
import { AddToCartButton } from '@/components/AddToCartButton';

export default async function ProductPage({ params }: { params: { id: string } }) {
  // Server handles data fetching
  const product = await getProduct(params.id);
  const relatedProducts = await getRelatedProducts(params.id);

  return (
    <div className="container mx-auto px-4 py-8">
      <div className="grid grid-cols-1 lg:grid-cols-3 gap-8">
        {/* Left column - All Server rendered */}
        <div className="lg:col-span-2">
          <ProductDetails product={product} />
          <ProductDescription product={product} />
          <ProductSpecs product={product} />
        </div>

        {/* Right column - Mix of Server and Client */}
        <div className="space-y-6">
          <div className="bg-white p-6 rounded-lg shadow-sm">
            <h2 className="text-2xl font-bold text-gray-900 mb-4">
              ${product.price}
            </h2>
            
            {/* Client Component for interactivity */}
            <AddToCartButton productId={product.id} />
          </div>

          <div className="bg-white p-6 rounded-lg shadow-sm">
            <h3 className="font-semibold text-gray-900 mb-4">
              Related Products
            </h3>
            
            {/* Server Component rendered list */}
            <RelatedProductList products={relatedProducts} />
          </div>
        </div>
      </div>
    </div>
  );
}

// Server Component
function ProductDetails({ product }: { product: any }) {
  return (
    <div>
      <img 
        src={product.imageUrl} 
        alt={product.name}
        className="w-full rounded-lg"
      />
      <h1 className="text-3xl font-bold text-gray-900 mt-6 mb-2">
        {product.name}
      </h1>
      <p className="text-gray-600">{product.tagline}</p>
    </div>
  );
}

// Server Component
function ProductDescription({ product }: { product: any }) {
  return (
    <div className="bg-white p-6 rounded-lg shadow-sm mt-6">
      <h2 className="text-xl font-semibold text-gray-900 mb-4">
        Description
      </h2>
      <p className="text-gray-600 leading-relaxed">{product.description}</p>
    </div>
  );
}

// Server Component
function ProductSpecs({ product }: { product: any }) {
  return (
    <div className="bg-white p-6 rounded-lg shadow-sm mt-6">
      <h2 className="text-xl font-semibold text-gray-900 mb-4">
        Specifications
      </h2>
      <dl className="grid grid-cols-2 gap-4">
        <dt className="text-gray-600">Weight</dt>
        <dd className="text-gray-900">{product.weight}</dd>
        <dt className="text-gray-600">Dimensions</dt>
        <dd className="text-gray-900">{product.dimensions}</dd>
      </dl>
    </div>
  );
}

// Server Component
function RelatedProductList({ products }: { products: any[] }) {
  return (
    <div className="space-y-4">
      {products.map((product) => (
        <div key={product.id} className="flex items-center space-x-4">
          <img 
            src={product.thumbnailUrl}
            alt={product.name}
            className="w-16 h-16 rounded"
          />
          <div>
            <h4 className="font-semibold text-gray-900">{product.name}</h4>
            <p className="text-gray-600">${product.price}</p>
          </div>
        </div>
      ))}
    </div>
  );
}
```

```tsx
// components/AddToCartButton.tsx (Client Component)
"use client";

import { useState } from 'react';

interface AddToCartButtonProps {
  productId: string;
}

export default function AddToCartButton({ productId }: AddToCartButtonProps) {
  const [isAdding, setIsAdding] = useState(false);
  const [added, setAdded] = useState(false);

  const handleAddToCart = async () => {
    setIsAdding(true);
    
    try {
      await fetch('/api/cart/add', {
        method: 'POST',
        headers: { 'Content-Type': 'application/json' },
        body: JSON.stringify({ productId }),
      });
      
      setAdded(true);
      setTimeout(() => setAdded(false), 2000);
    } catch (error) {
      console.error('Failed to add to cart:', error);
    } finally {
      setIsAdding(false);
    }
  };

  return (
    <button
      onClick={handleAddToCart}
      disabled={isAdding}
      className={`w-full px-6 py-3 rounded-lg font-semibold transition-colors ${
        added
          ? 'bg-green-600 text-white'
          : 'bg-blue-600 text-white hover:bg-blue-700'
      }`}
    >
      {isAdding ? 'Adding...' : added ? 'Added!' : 'Add to Cart'}
    </button>
  );
}
```

### Real-World Example: E-commerce Product Page

Here's a comprehensive example showing how to structure a complex page using both Server and Client Components effectively:

```tsx
// app/products/[id]/page.tsx
import { db } from '@/lib/db';
import { notFound } from 'next/navigation';
import { ProductGallery } from '@/components/ProductGallery';
import { ProductInfo } from '@/components/ProductInfo';
import { ProductReviews } from '@/components/ProductReviews';
import { RecentlyViewed } from '@/components/RecentlyViewed';

export default async function ProductPage({ params }: { params: { id: string } }) {
  const product = await db.product.findUnique({
    where: { id: params.id },
    include: {
      variants: true,
      reviews: {
        include: { user: true },
        orderBy: { createdAt: 'desc' },
        take: 5,
      },
      category: true,
    },
  });

  if (!product) {
    notFound();
  }

  return (
    <div className="min-h-screen bg-gray-50">
      {/* Server Component - Product main content */}
      <ProductContent product={product} />
      
      {/* Server Component - Reviews section */}
      <ProductReviews productId={product.id} reviews={product.reviews} />
      
      {/* Server Component - Related products */}
      <RelatedProducts categoryId={product.category.id} excludeId={product.id} />
    </div>
  );
}

async function ProductContent({ product }: { product: any }) {
  return (
    <div className="container mx-auto px-4 py-8">
      <div className="grid grid-cols-1 lg:grid-cols-2 gap-12">
        {/* Left: Gallery (Client Component for interactivity) */}
        <ProductGallery images={product.images} />
        
        {/* Right: Product info (mix of Server and Client) */}
        <ProductInfo 
          product={product}
          variants={product.variants}
        />
      </div>
    </div>
  );
}

async function RelatedProducts({ categoryId, excludeId }: { categoryId: string; excludeId: string }) {
  const relatedProducts = await db.product.findMany({
    where: {
      categoryId,
      id: { not: excludeId },
    },
    take: 4,
  });

  return (
    <div className="container mx-auto px-4 py-12">
      <h2 className="text-2xl font-bold text-gray-900 mb-6">
        Related Products
      </h2>
      <div className="grid grid-cols-1 md:grid-cols-2 lg:grid-cols-4 gap-6">
        {relatedProducts.map((product) => (
          <ProductCard key={product.id} product={product} />
        ))}
      </div>
    </div>
  );
}

function ProductCard({ product }: { product: any }) {
  return (
    <div className="bg-white rounded-lg shadow-sm overflow-hidden hover:shadow-md transition-shadow">
      <a href={`/products/${product.id}`}>
        <img 
          src={product.thumbnailUrl}
          alt={product.name}
          className="w-full h-48 object-cover"
        />
        <div className="p-4">
          <h3 className="font-semibold text-gray-900 mb-2">{product.name}</h3>
          <p className="text-gray-600 text-sm mb-2">{product.tagline}</p>
          <p className="text-lg font-bold text-gray-900">${product.price}</p>
        </div>
      </a>
    </div>
  );
}
```

## 6.5 Component Composition Patterns

Understanding how to compose Server and Client Components effectively is key to building maintainable Next.js applications.

### Pattern 1: Server Parent, Client Children

The most common pattern is to use Server Components as parents with Client Component children for interactivity.

```tsx
// Server Component
export default async function ServerParent() {
  const data = await fetchData();
  
  return (
    <div>
      <h1>Server Rendered Content</h1>
      <ClientChild data={data} />
    </div>
  );
}

// Client Component
"use client";

export function ClientChild({ data }: { data: any }) {
  const [state, setState] = useState(data);
  
  return (
    <div>
      <p>{state.name}</p>
      <button onClick={() => setState({ ...state, name: 'Updated' })}>
        Update
      </button>
    </div>
  );
}
```

### Pattern 2: Extracting Interactivity

Extract interactive parts into separate Client Components to keep most of your code as Server Components.

```tsx
// Server Component
export default function ProductList() {
  const products = await getProducts();
  
  return (
    <div>
      {products.map((product) => (
        <div key={product.id}>
          <ProductInfo product={product} />
          <AddToCartButton productId={product.id} />
        </div>
      ))}
    </div>
  );
}

// Server Component
function ProductInfo({ product }: { product: any }) {
  return (
    <div>
      <h2>{product.name}</h2>
      <p>{product.description}</p>
      <p>${product.price}</p>
    </div>
  );
}

// Client Component
"use client";

function AddToCartButton({ productId }: { productId: string }) {
  const [added, setAdded] = useState(false);
  
  const handleClick = () => {
    setAdded(true);
    // Add to cart logic
  };
  
  return (
    <button onClick={handleClick}>
      {added ? 'Added!' : 'Add to Cart'}
    </button>
  );
}
```

### Pattern 3: Passing Server Data to Client Components

Server Components can pass data to Client Components as props. This data is serialized and sent to the client.

```tsx
// Server Component
export default async function ServerWithData() {
  const user = await getCurrentUser();
  const posts = await getUserPosts(user.id);
  
  return (
    <div>
      <ClientDashboard user={user} posts={posts} />
    </div>
  );
}

// Client Component
"use client";

interface DashboardProps {
  user: { id: string; name: string; email: string };
  posts: { id: string; title: string; views: number }[];
}

export function ClientDashboard({ user, posts }: DashboardProps) {
  const [filter, setFilter] = useState('all');
  
  const filteredPosts = posts.filter(post => 
    filter === 'all' || post.views > 100
  );
  
  return (
    <div>
      <h1>Welcome, {user.name}!</h1>
      
      <div className="mb-4">
        <select value={filter} onChange={(e) => setFilter(e.target.value)}>
          <option value="all">All Posts</option>
          <option value="popular">Popular Posts</option>
        </select>
      </div>
      
      <ul>
        {filteredPosts.map(post => (
          <li key={post.id}>
            {post.title} - {post.views} views
          </li>
        ))}
      </ul>
    </div>
  );
}
```

### Pattern 4: Client Components Wrapping Server Components

Client Components can also render Server Components as children, though this is less common.

```tsx
// Client Component
"use client";

import { Suspense } from 'react';

interface TabContentProps {
  activeTab: string;
}

export function TabContent({ activeTab }: TabContentProps) {
  return (
    <div>
      {activeTab === 'overview' && (
        <Suspense fallback={<LoadingSpinner />}>
          <OverviewTab />
        </Suspense>
      )}
      {activeTab === 'details' && (
        <Suspense fallback={<LoadingSpinner />}>
          <DetailsTab />
        </Suspense>
      )}
      {activeTab === 'reviews' && (
        <Suspense fallback={<LoadingSpinner />}>
          <ReviewsTab />
        </Suspense>
      )}
    </div>
  );
}

// Server Components
async function OverviewTab() {
  const data = await getOverviewData();
  return <div>{/* Render overview */}</div>;
}

async function DetailsTab() {
  const data = await getDetailsData();
  return <div>{/* Render details */}</div>;
}

async function ReviewsTab() {
  const data = await getReviewsData();
  return <div>{/* Render reviews */}</div>;
}

function LoadingSpinner() {
  return <div className="animate-spin">Loading...</div>;
}
```

### Pattern 5: Compound Components

Use compound components to create flexible, composable UIs that mix Server and Client components.

```tsx
// Server Component
export default async function ProductPage() {
  const product = await getProduct();
  
  return (
    <ProductCard>
      <ProductCard.Image src={product.imageUrl} alt={product.name} />
      <ProductCard.Content>
        <ProductCard.Title>{product.name}</ProductCard.Title>
        <ProductCard.Price>${product.price}</ProductCard.Price>
        <ProductCard.Description>{product.description}</ProductCard.Description>
        <ProductCard.Actions>
          <AddToCartButton productId={product.id} />
        </ProductCard.Actions>
      </ProductCard.Content>
    </ProductCard>
  );
}

// Server Component
export function ProductCard({ children }: { children: React.ReactNode }) {
  return (
    <div className="bg-white rounded-lg shadow-sm overflow-hidden">
      {children}
    </div>
  );
}

ProductCard.Image = function Image({ src, alt }: { src: string; alt: string }) {
  return <img src={src} alt={alt} className="w-full" />;
};

ProductCard.Content = function Content({ children }: { children: React.ReactNode }) {
  return <div className="p-6">{children}</div>;
};

ProductCard.Title = function Title({ children }: { children: React.ReactNode }) {
  return <h3 className="text-xl font-bold text-gray-900 mb-2">{children}</h3>;
};

ProductCard.Price = function Price({ children }: { children: React.ReactNode }) {
  return <p className="text-2xl font-bold text-blue-600 mb-4">{children}</p>;
};

ProductCard.Description = function Description({ children }: { children: React.ReactNode }) {
  return <p className="text-gray-600 mb-6">{children}</p>;
};

ProductCard.Actions = function Actions({ children }: { children: React.ReactNode }) {
  return <div className="flex space-x-4">{children}</div>;
};

// Client Component
"use client";

function AddToCartButton({ productId }: { productId: string }) {
  const [added, setAdded] = useState(false);
  
  return (
    <button
      onClick={() => setAdded(true)}
      className="flex-1 bg-blue-600 text-white py-3 rounded-lg hover:bg-blue-700"
    >
      {added ? 'Added!' : 'Add to Cart'}
    </button>
  );
}
```

## 6.6 Passing Data Between Components

Data flow between Server and Client Components requires understanding serialization and what can be passed as props.

### Passing Serializable Data

Server Components can pass serializable data to Client Components:

```tsx
// Server Component
export default async function ServerPassingData() {
  const user = await db.user.findUnique({
    where: { id: '123' },
    select: {
      id: true,
      name: true,
      email: true,
      createdAt: true,
    },
  });
  
  // ✅ Passes serializable data
  return <UserProfile user={user} />;
}

// Client Component
"use client";

interface UserProps {
  user: {
    id: string;
    name: string;
    email: string;
    createdAt: Date;
  };
}

export function UserProfile({ user }: UserProps) {
  const [localState, setLocalState] = useState(user.name);
  
  return (
    <div>
      <h1>{localState}</h1>
      <p>{user.email}</p>
      <p>Joined: {user.createdAt.toLocaleDateString()}</p>
    </div>
  );
}
```

### What Can Be Passed as Props

✅ **Primitive Types:**
- strings, numbers, booleans, null, undefined

✅ **Arrays and Objects:**
- Plain arrays and objects (with serializable values)

✅ **Dates:**
- Date objects (automatically serialized)

✅ **Maps and Sets:**
- Maps and Sets (with serializable values)

✅ **BigInts:**
- BigInt values

❌ **Functions:**
- Cannot pass functions as props

❌ **Classes/Instances:**
- Cannot pass class instances

❌ **Symbols:**
- Cannot pass symbols

### Handling Non-Serializable Data

If you need to pass non-serializable data, use alternative approaches:

```tsx
// Server Component
export default async function ServerWithComplexData() {
  // ❌ Cannot pass this directly
  const complexObject = {
    id: '123',
    data: new Map(), // Non-serializable
    method: () => {}, // Non-serializable
  };
  
  // ✅ Pass serializable data instead
  const serializableData = {
    id: complexObject.id,
    data: Array.from(complexObject.data.entries()),
  };
  
  return <ClientComponent data={serializableData} />;
}

// Client Component
"use client";

export function ClientComponent({ data }: { data: any }) {
  // Reconstruct Map from array
  const map = new Map(data.data);
  
  return <div>{/* Use the reconstructed data */}</div>;
}
```

### Using Server Actions for Data Updates

Server Actions provide a way for Client Components to trigger server-side operations:

```tsx
// app/actions.ts
'use server';

import { db } from '@/lib/db';
import { revalidatePath } from 'next/cache';

export async function updateUserName(userId: string, name: string) {
  await db.user.update({
    where: { id: userId },
    data: { name },
  });
  
  // Revalidate the page to show updated data
  revalidatePath(`/users/${userId}`);
}

// Client Component
"use client";

import { useState } from 'react';
import { updateUserName } from '@/app/actions';

export function UserNameEditor({ userId, initialName }: { userId: string; initialName: string }) {
  const [name, setName] = useState(initialName);
  const [isUpdating, setIsUpdating] = useState(false);
  
  const handleSave = async () => {
    setIsUpdating(true);
    try {
      await updateUserName(userId, name);
      // Page will be revalidated automatically
    } catch (error) {
      console.error('Failed to update name:', error);
    } finally {
      setIsUpdating(false);
    }
  };
  
  return (
    <div>
      <input
        type="text"
        value={name}
        onChange={(e) => setName(e.target.value)}
        className="border rounded px-3 py-2"
      />
      <button
        onClick={handleSave}
        disabled={isUpdating}
        className="ml-2 bg-blue-600 text-white px-4 py-2 rounded"
      >
        {isUpdating ? 'Saving...' : 'Save'}
      </button>
    </div>
  );
}
```

## 6.7 Common Pitfalls and Solutions

Understanding common mistakes will help you avoid frustration and build better applications.

### Pitfall 1: Using Client Components Unnecessarily

**Problem:** Making everything a Client Component when Server Components would work better.

```tsx
// ❌ Bad - Unnecessary Client Component
"use client";

export default function StaticContent() {
  return (
    <div>
      <h1>About Us</h1>
      <p>We are a company that does things.</p>
    </div>
  );
}
```

**Solution:** Use Server Components for static content.

```tsx
// ✅ Good - Server Component
export default function StaticContent() {
  return (
    <div>
      <h1>About Us</h1>
      <p>We are a company that does things.</p>
    </div>
  );
}
```

### Pitfall 2: Forgetting "use client" Directive

**Problem:** Getting errors when trying to use hooks or event handlers without the "use client" directive.

```tsx
// ❌ Bad - Missing "use client"
export default function Counter() {
  const [count, setCount] = useState(0); // Error!
  return <button onClick={() => setCount(count + 1)}>{count}</button>;
}
```

**Solution:** Add "use client" at the top of the file.

```tsx
// ✅ Good - With "use client"
"use client";

export default function Counter() {
  const [count, setCount] = useState(0);
  return <button onClick={() => setCount(count + 1)}>{count}</button>;
}
```

### Pitfall 3: Importing Client Components into Server Components Incorrectly

**Problem:** Trying to use Client Components that have been incorrectly structured.

```tsx
// ❌ Bad - Importing Client Component that should be Server
import { ClientComponent } from './ClientComponent';

export default async function ServerComponent() {
  const data = await fetchData();
  return <ClientComponent data={data} />;
}
```

**Solution:** Ensure proper component boundaries.

```tsx
// ✅ Good - Proper separation
// components/ServerContent.tsx
export default async function ServerContent() {
  const data = await fetchData();
  return (
    <div>
      <h1>Server Content</h1>
      <ClientInteractivity data={data} />
    </div>
  );
}

// components/ClientInteractivity.tsx
"use client";

export function ClientInteractivity({ data }: { data: any }) {
  const [state, setState] = useState(data);
  return <div>{/* Interactive content */}</div>;
}
```

### Pitfall 4: Passing Non-Serializable Data

**Problem:** Trying to pass functions or non-serializable objects as props.

```tsx
// ❌ Bad - Passing function as prop
"use client";

export default function Parent() {
  const handleClick = () => console.log('clicked');
  return <Child onClick={handleClick} />;
}
```

**Solution:** Use Server Actions or pass data instead.

```tsx
// ✅ Good - Using Server Actions
'use server';

export async function handleClickAction() {
  console.log('clicked');
}

// components/Child.tsx
"use client";

import { handleClickAction } from '@/app/actions';

export function Child() {
  return <button onClick={() => handleClickAction()}>Click me</button>;
}
```

### Pitfall 5: Mixing Component Types Incorrectly

**Problem:** Trying to use Server Component features in Client Components.

```tsx
// ❌ Bad - Using server features in Client Component
"use client";

export default function BadClientComponent() {
  // Cannot do this
  const data = await fetchData(); // Error!
  return <div>{data.name}</div>;
}
```

**Solution:** Restructure to use Server Components for data fetching.

```tsx
// ✅ Good - Proper separation
export default async function GoodServerComponent() {
  const data = await fetchData();
  return <ClientDisplay data={data} />;
}

// components/ClientDisplay.tsx
"use client";

export function ClientDisplay({ data }: { data: any }) {
  const [localState, setLocalState] = useState(data.name);
  return <div>{localState}</div>;
}
```

### Pitfall 6: Not Optimizing Bundle Size

**Problem:** Converting too many components to Client Components, increasing bundle size.

```tsx
// ❌ Bad - Everything is Client Component
"use client";

export default function Page() {
  const [data, setData] = useState(null);
  
  useEffect(() => {
    fetchData().then(setData);
  }, []);
  
  return (
    <div>
      <Header />
      <MainContent data={data} />
      <Footer />
    </div>
  );
}
```

**Solution:** Keep as much as possible as Server Components.

```tsx
// ✅ Good - Server Components with Client Components only where needed
export default async function Page() {
  const data = await fetchData();
  
  return (
    <div>
      <Header />
      <MainContent data={data} />
      <Footer />
    </div>
  );
}

// components/MainContent.tsx
"use client";

export function MainContent({ data }: { data: any }) {
  const [localState, setLocalState] = useState(null);
  return <div>{/* Interactive content */}</div>;
}
```

## Key Takeaways from Chapter 6

1. **Mental Model Shift**: Next.js introduces Server Components that render on the server by default, sending only HTML to the client, while Client Components (marked with "use client") render on the client like traditional React components.

2. **Server Components by Default**: All components are Server Components unless marked otherwise. They can access server resources directly, don't send JavaScript to the client, and improve performance and SEO.

3. **Client Components with "use client"**: Client Components require the "use client" directive and are used for interactivity, browser APIs, React hooks, and event handling.

4. **When to Use Each**: Use Server Components for data fetching, static content, sensitive operations, and SEO-critical pages. Use Client Components for user interactions, browser APIs, state management, and event listeners.

5. **Component Composition**: Mix Server and Client Components strategically, using Server Components as parents with Client Component children, extracting interactivity into separate Client Components, and passing serializable data between components.

6. **Passing Data**: Server Components can pass serializable data (primitives, arrays, objects, dates, maps, sets) to Client Components as props, but cannot pass functions or non-serializable objects.

7. **Common Pitfalls**: Avoid unnecessary Client Components, remember the "use client" directive, properly structure component boundaries, don't pass non-serializable data, and optimize bundle size by keeping most components as Server Components.

## Coming Up Next

**Chapter 7: Styling Approaches**

Now that you understand Server and Client Components, it's time to make your applications look great! In Chapter 7, we'll explore various styling approaches in Next.js, including global CSS, CSS Modules, Tailwind CSS, CSS-in-JS solutions, and modern styling patterns. You'll learn how to implement responsive designs, handle dark mode, and create polished, professional UIs.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='5. layouts_and_pages.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='7. styling_approaches.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
